# Desafio: Engenharia de Dados

## Task1

Leia o arquivo de texto ​ wordcount.txt ​ e conte as palavras que contém até 10 letras. Conte
também quantas palavras com mais de 10 letras existem no texto.
Dataset: ​ https://storage.googleapis.com/luizalabs-hiring-test/wordcount.txt
Exemplo do dataset final:
[('two', 2),
('behold', 1),
('itself', 3),
(‘MAIORES QUE 10’, 3)]

In [1]:
# Biblioteca Task1
import findspark
findspark.init('/home/lucas/spark-3.0.0-preview-bin-hadoop2.7')

import pyspark
from pyspark.sql import SparkSession
import requests
import re

In [2]:
spark = SparkSession.builder.appName('desafio').getOrCreate()

In [3]:
# Leitura do texto
url1 = 'https://storage.googleapis.com/luizalabs-hiring-test/wordcount.txt'
r = requests.get(url1)
text_task_1 = r.text

In [4]:
print("Tamanho do texto: {} caracteres \n".format(len(text_task_1)))
# Para visualizar um pedaço do arquivo
print(text_task_1[0:100])

Tamanho do texto: 18436 caracteres 

henDRERIt. MoNTEs PURus lUCtUS DIcTuM, eST Mattis ESt phasELLUS DiGNISsiM rHoNcUS cuBIlIa sIT nunC a


In [5]:
def remove_punctuation(text):
    ext = re.sub('\\t','',text)
    text = re.sub('\\n','',text)
    text = re.sub('[^A-Za-z0-9 ]+', '', text)
    return text.lower()

In [6]:
# Detalhe: neste caso estou considerando que a identidade da palavra
# não é case sensitive, ou seja, "Ou" é igual a "OU" e "oU".
clean_text = remove_punctuation(text_task_1)

In [7]:
clean_text

'hendrerit montes purus luctus dictum est mattis est phasellus dignissim rhoncus cubilia sit nunc at interdum vivamus luctus ante ac amet elementum congue justo eget egestas vitae per lacus sociis purus quisque luctus eleifend inceptos condimentum etiam vel augue vivamus proin eget placerat conubia odio inceptos feugiat etiam venenatis class lacus ultrices convallis aenean diam donec in sociis neque ac dignissim at ultrices nisl semper netus cubilia nunc ante porttitor aliquet proin eros viverra sodales netus integer auctor tortor ultricies cubilia netus posuere ipsum quam montes sociosqu vivamus condimentum in lectus nascetur eu etiam nec tempor posuere sagittis rutrum luctus facilisis feugiat turpis platea ut eros porttitor viverra ligula ipsum etiam placerat elementum sem semper nec fringilla dictumst eget sit malesuada sagittis arcu justo cum odio ante lectus potenti nibh amet vel netus penatibus vivamus rutrum orci sollicitudin commodo placerat nulla morbi pulvinar conubia sodales

In [8]:
word_list = clean_text.split()

In [9]:
word_list

['hendrerit',
 'montes',
 'purus',
 'luctus',
 'dictum',
 'est',
 'mattis',
 'est',
 'phasellus',
 'dignissim',
 'rhoncus',
 'cubilia',
 'sit',
 'nunc',
 'at',
 'interdum',
 'vivamus',
 'luctus',
 'ante',
 'ac',
 'amet',
 'elementum',
 'congue',
 'justo',
 'eget',
 'egestas',
 'vitae',
 'per',
 'lacus',
 'sociis',
 'purus',
 'quisque',
 'luctus',
 'eleifend',
 'inceptos',
 'condimentum',
 'etiam',
 'vel',
 'augue',
 'vivamus',
 'proin',
 'eget',
 'placerat',
 'conubia',
 'odio',
 'inceptos',
 'feugiat',
 'etiam',
 'venenatis',
 'class',
 'lacus',
 'ultrices',
 'convallis',
 'aenean',
 'diam',
 'donec',
 'in',
 'sociis',
 'neque',
 'ac',
 'dignissim',
 'at',
 'ultrices',
 'nisl',
 'semper',
 'netus',
 'cubilia',
 'nunc',
 'ante',
 'porttitor',
 'aliquet',
 'proin',
 'eros',
 'viverra',
 'sodales',
 'netus',
 'integer',
 'auctor',
 'tortor',
 'ultricies',
 'cubilia',
 'netus',
 'posuere',
 'ipsum',
 'quam',
 'montes',
 'sociosqu',
 'vivamus',
 'condimentum',
 'in',
 'lectus',
 'nascetur'

In [10]:
def word_counter(lista,num_max):
    count_words = {}
    key_max = 'MAIORES QUE ' + str(num_max)
    count_words = {key_max:0}
    for word in lista:
        if len(word) <=num_max:
            if word in count_words:
                count_words[word] +=1
            else:
                count_words[word] =1
        else:
            count_words[key_max] +=1
    return [(key,value) for (key,value) in count_words.items()]

In [11]:
objective1 = sorted(word_counter(word_list,10), key= lambda x: x[1])

In [12]:
import csv
with open('words_list.csv', 'w', newline='') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerows(objective1)

# Task 2

- Leia o arquivo pedidos.csv 
- Agrupe todos os cliente que fizeram mais de 2 compras nos dias de black friday dos últimos três anos. 
- Filtre todos os clientes que são menores de 30 anos e coloque numa lista TODOS os códigos de pedido e a data em que foram efetuados. 
- Adicione também a idade do cliente. 

Dataset: ​ https://storage.googleapis.com/luizalabs-hiring-test/clientes_pedidos.csv 


In [13]:
# Leitura do Arquivo:
url2 = 'https://storage.googleapis.com/luizalabs-hiring-test/clientes_pedidos.csv'
from pyspark import SparkFiles
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DateType

spark.sparkContext.addFile(url2)
table = spark.read.csv("file://" + SparkFiles.get('clientes_pedidos.csv'), header=True)
print(table.schema)

StructType(List(StructField(codigo_pedido,StringType,true),StructField(codigo_cliente,StringType,true),StructField(data_nascimento_cliente,StringType,true),StructField(data_pedido,StringType,true)))


https://medium.com/luizalabs/as-armadilhas-do-spark-101-d13a3296dcd9

In [14]:
schema = StructType([
    StructField("codigo_pedido",StringType(),True),
    StructField("codigo_cliente",StringType(),True),
    StructField("data_nascimento_cliente",StringType(),True),
    StructField("data_pedido",StringType(),True)])

table = spark.read.csv("file://" + SparkFiles.get('clientes_pedidos.csv'), header=True, schema=schema)

In [15]:
table.printSchema()

root
 |-- codigo_pedido: string (nullable = true)
 |-- codigo_cliente: string (nullable = true)
 |-- data_nascimento_cliente: string (nullable = true)
 |-- data_pedido: string (nullable = true)



In [16]:
table.select("data_nascimento_cliente").head()

Row(data_nascimento_cliente='1985-12-04 00:00:00')

In [17]:
table.select("data_pedido").head()

Row(data_pedido='1542974527')

In [18]:
from pyspark.sql.functions import (to_date,to_timestamp,
                                    unix_timestamp,
                                  from_unixtime,
                                  lit, current_date, datediff,
                                  collect_list, arrays_zip,
                                  concat_ws)

In [19]:
new_table = table.withColumn('d_nascimento', to_date(table.data_nascimento_cliente).alias("to_date"))


In [20]:
new_table.select("d_nascimento").show()

+------------+
|d_nascimento|
+------------+
|  1985-12-04|
|  1979-11-14|
|  1989-07-25|
|  1953-12-14|
|  1985-05-03|
|  1980-04-16|
|  1991-11-18|
|  1974-01-04|
|  1985-04-18|
|  1981-09-15|
|  1996-04-17|
|  1970-11-15|
|  1964-10-12|
|  1990-06-25|
|  1979-09-30|
|  1958-05-13|
|  1976-05-18|
|  1988-01-13|
|  1993-08-10|
|  1978-11-20|
+------------+
only showing top 20 rows



In [21]:
table.select('data_pedido').describe().show()

+-------+--------------------+
|summary|         data_pedido|
+-------+--------------------+
|  count|              297309|
|   mean|1.5287494219447646E9|
| stddev|1.5643343589912213E7|
|    min|          1509503453|
|    max|          1543622000|
+-------+--------------------+



Referência: https://sparkbyexamples.com/spark/spark-sql-date-and-time-functions/

Aparentemente estamos lidando com a contagem em segundos com relação ao marco UNIX (1970-01-01 00:00:00 UTC).

In [22]:
new_table = new_table.withColumn('d_pedido',to_date(from_unixtime(table.data_pedido)))



In [23]:
new_table.select('d_pedido').show()

+----------+
|  d_pedido|
+----------+
|2018-11-23|
|2018-11-23|
|2018-11-23|
|2018-11-23|
|2018-11-23|
|2018-11-23|
|2018-11-23|
|2018-11-23|
|2018-11-23|
|2018-11-23|
|2018-11-23|
|2018-11-23|
|2018-11-23|
|2018-11-23|
|2018-11-23|
|2018-11-23|
|2018-11-23|
|2018-11-23|
|2018-11-23|
|2018-11-23|
+----------+
only showing top 20 rows



In [24]:
new_table.head()

Row(codigo_pedido='bc8b03a005d5bf742fc7290db1b218de', codigo_cliente='b07af86a4a68707373856bcc3946583f', data_nascimento_cliente='1985-12-04 00:00:00', data_pedido='1542974527', d_nascimento=datetime.date(1985, 12, 4), d_pedido=datetime.date(2018, 11, 23))

In [25]:
new_table.printSchema()

root
 |-- codigo_pedido: string (nullable = true)
 |-- codigo_cliente: string (nullable = true)
 |-- data_nascimento_cliente: string (nullable = true)
 |-- data_pedido: string (nullable = true)
 |-- d_nascimento: date (nullable = true)
 |-- d_pedido: date (nullable = true)



Black friday dos últimos três anos:
- 2018: 23 de novembro
- 2017: 24 de novembro
- 2016: 25 de novembro

In [26]:
black_fridays = ("2018-11-23", "2017-11-24", "2016-11-25")

# filtrando dias de pedido correspondentes à Black Friday
result = new_table.filter((new_table.d_pedido == (lit(black_fridays[0])))|(new_table.d_pedido ==lit(black_fridays[1]))|(new_table.d_pedido ==lit(black_fridays[2]))).groupBy('codigo_cliente').agg({'codigo_pedido':'count'})

In [27]:
result.printSchema()

root
 |-- codigo_cliente: string (nullable = true)
 |-- count(codigo_pedido): long (nullable = false)



In [28]:
result.columns

['codigo_cliente', 'count(codigo_pedido)']

In [29]:
clientes_black = result.filter(result['count(codigo_pedido)']>2).select('codigo_cliente')

In [30]:
clientes_black.show()

+--------------------+
|      codigo_cliente|
+--------------------+
|b67ef7abecc0a8e88...|
|28688f66084a7f1de...|
|3f89d915a06a3d01e...|
|c371799c2befffb67...|
|e7c52e68263476a2d...|
|3bfcd49a281054bbf...|
|f184a197ec54c7eb1...|
|1375a4e01d4811249...|
|3531fd9696a342b74...|
|4cf0d3732731a0653...|
|c512a4d48ee9388e1...|
|057ae5d7ef3fcdd74...|
|d2b6a6676c81b7e84...|
|e40809b3ff1805b54...|
|c3f1cb573d199ae93...|
|18d6c857acbfb21b5...|
|3860f681456fae15d...|
|fcf0d288e3488a3ab...|
|01b9f95dce03e2382...|
|81ba3bb6df12e897c...|
+--------------------+
only showing top 20 rows



In [31]:
# Restringindo os clientes aos que fizeram mais de 2 compras ao longo das três últimas Black Fridays 

compras_clientes_black = clientes_black.join(new_table, clientes_black.codigo_cliente == new_table.codigo_cliente, "inner")
compras_clientes_black = compras_clientes_black.drop(new_table["codigo_cliente"])

In [32]:
compras_clientes_black = compras_clientes_black.withColumn('idade', ((datediff(current_date(),compras_clientes_black.d_nascimento))/365).cast('integer'))

In [33]:
compras_clientes_black.select('idade').show(5)

+-----+
|idade|
+-----+
|   49|
|   49|
|    8|
|   49|
|   49|
+-----+
only showing top 5 rows



In [34]:
compras_clientes_black_under30 = compras_clientes_black.filter(compras_clientes_black['idade']<30)

https://stackoverflow.com/questions/37440373/spark-dataframe-aggregate-column-values-by-key-into-list

In [35]:
compras_clientes_black_under30 = compras_clientes_black_under30.withColumn("d_pedido_2", compras_clientes_black_under30.d_pedido.cast(StringType())).drop("d_pedido").withColumnRenamed("d_pedido_2", "d_pedido")


pedido = (compras_clientes_black_under30.groupBy(compras_clientes_black_under30["codigo_cliente"])
     .agg(collect_list("codigo_pedido").alias("pedidos"),
        collect_list("d_pedido").alias("data"))
     .withColumn("pedidos", arrays_zip("pedidos","data")).drop("data"))



#pedido = (compras_clientes_black_under30.groupBy(compras_clientes_black_under30["codigo_cliente"])
#     .agg(collect_list("codigo_pedido").alias("pedidos"),
#        collect_list("d_pedido").alias("data"))
#     .withColumn("pedidos", arrays_zip("pedidos","data")).drop("data"))


#(df
#  .groupby("id")
#  .agg(F.collect_set("code"),
#       F.collect_list("name"))
#  .show())

#df2 = df.groupBy("name").agg(
#  collect_list(col("food")) as "food",
#  collect_list(col("price")) as "price" 
#).withColumn("food", zipper(col("food"), col("price"))).drop("price")

In [36]:
pedido.show()

+--------------------+--------------------+
|      codigo_cliente|             pedidos|
+--------------------+--------------------+
|3bfcd49a281054bbf...|[[a071669a5000ce0...|
|18d6c857acbfb21b5...|[[aede83e21d3683a...|
|3860f681456fae15d...|[[66c1ea9f1aa3b25...|
|a3c3bc3ebac103447...|[[07089f306713afa...|
|d221905b7af7b56c0...|[[82c2a7a825ac040...|
|4112bda7877566650...|[[08d28852abb0610...|
|047f50ac99e51749f...|[[89419f4a8b4d993...|
|70a35a3d4743b9503...|[[88bebb01617d795...|
|22275184edd41a8a9...|[[44328873b865655...|
|3224ac7dee3bd6196...|[[110fdb292cc6b2c...|
|a61b2a2669b043efb...|[[4a499631d7360c3...|
|e357a3944a07df38f...|[[9ebfc7efabf4d41...|
|77547c27288c4114e...|[[adbdd9fb05ecd13...|
|9436b0e2e66029690...|[[eb9f57c8fd92752...|
|6ac4d4625eaaf12da...|[[d2973feaa1be74d...|
|48bd44c049ba3ff9d...|[[122fb9734cb8e79...|
|d639cb78545b7c00d...|[[787f7bb91d3e155...|
|76fa9d1303b605161...|[[df5b474943a7c14...|
|e6302fd4bc1b9a8c8...|[[9b17d4b5254b8d0...|
|9ba43b1b796103b4f...|[[ae5f74bb

In [37]:


visual = compras_clientes_black_under30.join(pedido, compras_clientes_black_under30.codigo_cliente == pedido.codigo_cliente,"inner").drop(pedido["codigo_cliente"])
#
visual = visual.withColumn("d_pedido_2", visual.d_pedido.cast(StringType())).drop("d_pedido").withColumnRenamed("d_pedido_2", "d_pedido")
#
visual = visual.join(result, visual.codigo_cliente == result.codigo_cliente).drop(result["codigo_cliente"])
visual = visual.select(['codigo_cliente','count(codigo_pedido)','idade','pedidos'])
visual = visual.dropDuplicates()
visual.show()

+--------------------+--------------------+-----+--------------------+
|      codigo_cliente|count(codigo_pedido)|idade|             pedidos|
+--------------------+--------------------+-----+--------------------+
|3bfcd49a281054bbf...|                   3|   19|[[a071669a5000ce0...|
|18d6c857acbfb21b5...|                   3|   29|[[aede83e21d3683a...|
|3860f681456fae15d...|                   3|   19|[[66c1ea9f1aa3b25...|
|a3c3bc3ebac103447...|                   3|    9|[[07089f306713afa...|
|d221905b7af7b56c0...|                   3|   27|[[82c2a7a825ac040...|
|4112bda7877566650...|                   3|   23|[[08d28852abb0610...|
|047f50ac99e51749f...|                   3|   25|[[89419f4a8b4d993...|
|70a35a3d4743b9503...|                   3|   23|[[88bebb01617d795...|
|22275184edd41a8a9...|                   3|   25|[[44328873b865655...|
|3224ac7dee3bd6196...|                   3|   19|[[110fdb292cc6b2c...|
|a61b2a2669b043efb...|                   3|   23|[[4a499631d7360c3...|
|e357a

In [38]:
visual.select('pedidos').show(10, False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|pedidos                                                                                                                                                                                         |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[[a071669a5000ce07ad709886c8109906, 2018-11-23], [ca2360f9f20482f1f0d851a15ae67498, 2018-11-23], [cb422c641730320669dc49b88c069a00, 2018-11-23]]                                                |
|[[aede83e21d3683a716106a85d208ded9, 2018-11-23], [fa32e1d7d905159afec553b506dc4394, 2018-11-23], [3a6abfc49a47d5d30d06747b5e781681, 2018-11-23], [b9f4d17e40a03154ada5c1ed1a1d87f4, 2017-11-11]]|
|[[66c1ea9f1aa3b2544d5e9b

In [39]:
print(visual.schema)

StructType(List(StructField(codigo_cliente,StringType,true),StructField(count(codigo_pedido),LongType,false),StructField(idade,IntegerType,true),StructField(pedidos,ArrayType(StructType(List(StructField(pedidos,StringType,true),StructField(data,StringType,true))),false),true)))


In [40]:
#visual.select('pedidos')

In [41]:
#from pyspark.sql.functions import udf, col

#join_udf = udf(lambda x: "[" + ",".join(str(x)) + "]", StringType())
#visual.withColumn("pedidos_str", join_udf(visual.pedidos)).show()

#visual = visual.drop(visual['pedidos'])

In [42]:
#visual.show()

In [43]:
visual.write.json('pedidos.json')